# Set cluster, accounting group, script, etc.
Some variables can either be set in this notebook, or in environment variables (`JOBSERVANT_CLUSTER` and `JOBSERVANT_ACCOUNTING_GROUP`).

It is assumed that this notebook is running in an SSH agent with keys loaded, and some required python packages are installed -- see the README.md for more details.

In [ ]:
import os

cluster_name = os.environ.get('JOBSERVANT_CLUSTER') or 'cluster.name.here.please'

accounting_group = os.environ.get('JOBSERVANT_ACCOUNTING_GROUP') or 'accounting-group-here-please'
job_params = {
    'account': accounting_group,
    'time': '00:01:30',

    # Can ask for additional stuff, e.g.
    # 'mem': "1500000"
}

# This is what we will run on the cluster:
job_script = """
hostname
echo 'Hello World'
sleep 60
"""

# Importing `jobservant` classes

In [ ]:
import os
import sys

# Find jobservant
this_dir = os.path.dirname(os.path.realpath('.'))
sys.path.append(this_dir)

# Load / reload jobservant
from jobservant.cluster_account import ClusterAccount
from jobservant.jupyter.job_presenter import JobPresenter

# Connect to the cluster and submit a job

We first create a `ClusterAccount` object, then ask it to create and submit a job for us on the cluster. A `ClusterJob` object is returned that we can use to monitor and manipulate our remote job on the cluster.

In [ ]:
# Try log_level='debug' for lotsa output
# Set username='whomever' if your username isn't automatically detected
cluster_account = ClusterAccount(server=cluster_name, log_level='info')

job = cluster_account.submit_job(text=job_script, **job_params)

# Monitor progress of job
We'll create a `JobPresenter` object for our job and run the `progress` action on it to monitor the jobs adventure through the scheduling system. This runs in the background so other cells can run while the output is updated (configured to update every 20 seconds).

Note that we've asked for the `include_output` option, so the output of the job will be displayed on job completion.

In [ ]:
job_presenter = JobPresenter(job)
job_presenter.progress(include_output=True)

# Low-level job checking

The following commands can be run outside of a Jupyter notebook to inspect the current state of a job (e.g., in a script or in a console).

In [ ]:
print(job.jobid, '\n')
print(job.work_directory, '\n')
print(job.status(), '\n')
print(job.queue_status_hash(), '\n')
print(job.efficiency_hash(), '\n')

# High-level job checking

The `JobPresenter` class has a `status_table` method for viewing a nice summary of a job.

This output won't auto-update, and only shows the current state.

In [ ]:
job_presenter.status_table()

# Check job output manually
Fetches the 'out' file created by the scheduler, throws an error if the job isn't finished

In [ ]:
job.fetch_output()

# Other actions

```
# Fetch stderr output at end of job
error = job.fetch_error()

# Fetch contents of any file in work directory on the cluster
foo_buffer = job.fetch_file('foo.txt')

# Create a file in the work directory on the cluster
content = "I am content for the file hello.txt"
job.create_remote_file('hello.txt', content)

# Cancel job
job.cancel()

# Clean up (delete) the job directory on the cluster
job.cleanup()
```